In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [ ]:
# 텐서플로우에서 제공하는 MNIST데이터를 사용하겠다
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지 데이터의 픽섹 범위는 0~255이므로 255로 나누어 0~1사이 범위로 변경 255.0은 샘플 데이터를 정수에서 부동소수점 숫자로 바꾸어준다.
x_train, x_test = x_train / 255.0, x_test / 255.0

#  차원 추가 - 아래에서 train_df를 만들때 y_train 값을 추가할 차원 필요 - 우리는 아래에서 Conv2D의 input_shape는 'batch_shape + (rows, cols, channels)' 이기 때문이다.
print(x_train.shape)
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')
print(x_train.shape)


(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
# 앞에서 shuffle에 들어가는 숫자 만큼의 데이터를 뽑은 후 랜덤 배치를 구성(배치에 사용된 만큼 뒤에 데이터를 추가하여 새로운 배치를 뽑기  반복)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(10000).batch(32)


In [ ]:
for i,i2 in train_ds:
  print(i.shape)
  print(i2.shape)
  break

(32, 28, 28, 1)
(32,)


In [ ]:
# 위의 동작을 테스트 해보기 위한 코드
test = tf.data.Dataset.from_tensor_slices(
    [1,2,3,4,5,6,7,8,9]).shuffle(3).batch(5)
for k in test:
    print(k)
    break

tf.Tensor([3 1 2 4 7], shape=(5,), dtype=int32)


In [ ]:
class mymodel(Model):
    def __init__(self):
        super(mymodel, self).__init__() 
        # 2차원 conv레이어 정의
        self.conv1 = Conv2D(32, 3, activation='relu')
        # flatten 레이어 정의 : 1차원 텐서로 바꾸어주기 위함
        self.flatten = Flatten()
        # 위의 값을 dense레이어를 통과시켜 128가지로 나눈다.
        self.d1 = Dense(128, activation='relu')
        # 위의 값을 dense레이어를 통과시켜 10가지로 나눈다.
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)

        return self.d2(x)
    
model = mymodel()

In [ ]:
# loss_func의 from_logits가 True인 이유는 위의 모델에서 마지막에 softmax와 같은 함수를 통과하지 않아 확률형값이 아니기에 사용(마지막에 softmax가 있었다면 from_logits=False)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acuuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acuuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    # 아래의 pred와 loss를 기록하기 위해 GradientTape를 사용
    with tf.GradientTape() as tape:
        pred = model(images, training=True)
        loss = loss_func(labels, pred)
    # loss 와 모델의 변수들을 통해 gradient를 계산한다.
    gradients = tape.gradient(loss, model.trainable_variables)
    # 계산된 gradient과 optimizer 함수를 통해 최적화된 모델의 변수들을 업데이트 해준다. 
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, pred)


In [ ]:
# 테스트 단계의 경우 위와 같은 loss나 optimizer의 기능이 필요없고 dropout이 일어나지 않도록 training=False를 지정해준다.
@tf.function
def test_step(images, labels):
    pred = model(images, training=False)
    t_loss = loss_func(labels, pred)
    
    test_loss(t_loss)
    test_accuracy(labels, pred)

In [ ]:
epochs = 5
for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
#     print('?')
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
#         print('?')
    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.133869931101799, Accuracy: 95.961669921875, Test Loss: 0.05833699554204941, Test Accuracy: 98.0999984741211
Epoch 2, Loss: 0.04224833473563194, Accuracy: 98.69667053222656, Test Loss: 0.05459906905889511, Test Accuracy: 98.16999816894531
Epoch 3, Loss: 0.022207919508218765, Accuracy: 99.2699966430664, Test Loss: 0.06738417595624924, Test Accuracy: 97.94999694824219
Epoch 4, Loss: 0.013740891590714455, Accuracy: 99.55166625976562, Test Loss: 0.057486020028591156, Test Accuracy: 98.41999816894531
Epoch 5, Loss: 0.009961310774087906, Accuracy: 99.6433334350586, Test Loss: 0.057501692324876785, Test Accuracy: 98.47999572753906
